<a href="https://colab.research.google.com/github/enok/flowpredict/blob/main/flowpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Free Cash Flow Forecasting

<small><div style="text-align: right">The data will consist of quarterly reports spanning from 2012 to 2022.</div></small>
<small><div style="text-align: right">The forecasting will be from 2023 to 2025.</div></small>

## 1. Downloading and loading stock market data from the CVM website.

### Companies to be analised


| Company       | Ticker    | CVM Code  |
| :---          | :----     | ----:     |
| CPFL Energia  | CPFE3     | 18660     |
| Cemig         | CMIG4     | 2453      |
| Eletrobras    | ELET6     | 2437      |
| Enev          | ENEV3     | 21237     |
| Taesa         | TAEE11    | 20257     |

In [1]:
companies = {
    18660: {"name": "CPFL Energia", "ticker": "CPFE3"},
    2453:  {"name": "Cemig", "ticker": "CMIG4"},
    2437:  {"name": "Eletrobras", "ticker": "ELET6"},
    21237: {"name": "Enev", "ticker": "ENEV3"},
    20257: {"name": "Taesa", "ticker": "TAEE11"}
}

print(f'companies: {companies}\n')

companies: {18660: {'name': 'CPFL Energia', 'ticker': 'CPFE3'}, 2453: {'name': 'Cemig', 'ticker': 'CMIG4'}, 2437: {'name': 'Eletrobras', 'ticker': 'ELET6'}, 21237: {'name': 'Enev', 'ticker': 'ENEV3'}, 20257: {'name': 'Taesa', 'ticker': 'TAEE11'}}



### Accounts selected

Based on:<br>
Investiment Valuation Tools and Techniques for Determining the Value of Any Asset (Aswath Damodaran)<br>
    CHAPTER 14 - Free Cash Flow to Equity Discount Models<br><br>

*Free Cash Flows to Equity (FCFE) = <br>
	Net income - (Capital expenditures - Depreciation) <br>
	- (Change in noncash working capital) <br>
	+ (New debt issued - Debt repayments)*<br><br>


*Fluxo de Caixa Livre para os Acionistas (FCLPA) = <br>
Lucro Líquido<br>
 \- (Investimentos em Ativos Fixos - Depreciação)<br>
 \- (Variação no Capital de Giro, excluindo Caixa)<br>
 \+ (Novas Dívidas Emitidas - Pagamentos de Dívida)*<br><br>

*FCFE = <br>
  3.11 (Lucro/Prejuizo Consolidado do Periodo do DRE)<br>
  \- (Δ 1.02.03 (Imobilizado) - 7.04.01 (Depreciacao, Amortizacao e Exaustao da DVA))<br>
  \- (Δ 1.01 (Ativo Circulante) - Δ 1.01.01 (Caixa e Equivalentes de Caixa) -  Δ 2.01 (Passivo Circulante))<br>
  \+ (Δ 2.01.04(Emprestimos e Financiamentos Circulantes) + Δ 2.02.01(Emprestimos e Financiamentos Nao Circulantes))*<br><br>



**BPA**

| Code       | Description                                           |
| :---       | :----                                                 |
| 1.01       | Ativo Circulante                                      |
| 1.01.01    | Caixa e Equivalentes de Caixa                         |
| 1.02.03    | Imobilizado                                           |

<br>                                                                

**BPP**

| Code       | Description                                           |
| :---       | :----                                                 |
| 2.01       | Passivo Circulante                                    |
| 2.01.04    | Empréstimos e Financiamentos (Passivo Circulante)     |
| 2.02.01    | Empréstimos e Financiamentos (Passivo Nao Circulante) |

<br>                                                                
                                                                    
**DRE**

| Code       | Description                                           |
| :---       | :----                                                 |
| 3.11       | Lucro/Prejuízo Consolidado do Período                 |

<br>                                                                

**DVA**
                                                     
| Code       | Description                                           |
| :---       | :----                                                 |
| 7.04.01    | Depreciação, Amortização e Exaustão                   |


<small><div style="text-align: right">Source: <a>https://www.rad.cvm.gov.br/ENET/documentos/EspecificacaoArquivoXML_InformacoesTrimestrais.xlsx<a></div></small>

In [2]:
file_types = ['BPA', 'BPP', 'DRE', 'DVA']
print(f'file_types selected: {file_types}')

account_to_be_kept = ['1.01','1.01.01','1.02.03',
                    '2.01','2.01.04','2.02.01',
                    '3.11',
                    '7.04.01']
print(f'accounts selected: {account_to_be_kept}')

file_types selected: ['BPA', 'BPP', 'DRE', 'DVA']
accounts selected: ['1.01', '1.01.01', '1.02.03', '2.01', '2.01.04', '2.02.01', '3.11', '7.04.01']


### Interval of years

In [3]:
years = range(2012, 2023)

### Installing libs

In [4]:
%pip install requests beautifulsoup4
%pip install chardet
%pip install files
%pip install chardet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Importing libs

In [5]:
import pandas as pd
import numpy as np
import re
import os
import requests
import zipfile
import chardet
from urllib.parse import urljoin
from bs4 import BeautifulSoup


### Setting Display

In [6]:
#pd.set_option('display.max_columns', 10)
pd.reset_option('display.max_columns')
pd.set_option('display.width', 200)
#pd.reset_option('display.width')

### Downloading balance sheets files

In [7]:
# Function to check if a file's encoding is UTF-8
def is_utf8(file_path):
    try:
        with open(file_path, 'rb') as file:
            raw_data = file.read()
            result = chardet.detect(raw_data)
            return result['encoding'] == 'utf-8'
    except Exception:
        return False

# URL of the website containing the ZIP files
base_url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"

# Create a directory to save the downloaded files
download_dir = "downloaded_files"
os.makedirs(download_dir, exist_ok=True)

# Define the years you want to download
years_to_download = set(str(year) for year in years)

# Send an HTTP GET request to the URL
response = requests.get(base_url)
response.raise_for_status()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all links on the page
links = soup.find_all("a")

# Iterate through the links and download ZIP files for the specified years
for link in links:
    file_url = urljoin(base_url, link["href"])

    # Example: Assuming ZIP files are named like "itr_cia_aberta_YEAR.zip"
    if file_url.endswith(".zip"):
        zip_file_name = os.path.basename(file_url)

        # Extract the year from the ZIP file name
        year_part = zip_file_name.split("_")[-1].split(".")[0]

        # Check if the ZIP file is from a year within the specified range
        if year_part in years_to_download:
            # Check if the ZIP file already exists
            if not os.path.exists(os.path.join(download_dir, zip_file_name)):
                print(f"Downloading: {zip_file_name}")
                with open(os.path.join(download_dir, zip_file_name), "wb") as file:
                    file_response = requests.get(file_url)
                    file.write(file_response.content)
            else:
                print(f"File already exists: {zip_file_name}")

# Process the downloaded ZIP files (extract, remove "ind" files, and convert)
for zip_file_name in os.listdir(download_dir):
    if zip_file_name.endswith(".zip"):
        zip_file_path = os.path.join(download_dir, zip_file_name)
        zip_subdir = os.path.splitext(zip_file_name)[0]  # Use ZIP file name without extension as subdirectory name
        zip_subdir_path = os.path.join(download_dir, zip_subdir)

        # Check if the ZIP file has already been extracted
        if not os.path.exists(zip_subdir_path):
            print(f"Unzipping: {zip_file_name} -> {zip_subdir_path}")
            with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(zip_subdir_path)

            # Remove files that are not in file_types
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    csv_file_type = os.path.splitext(file_name)[0][15:19].replace("_", "")
                    print(f"CSV type: {csv_file_type}")
                    if csv_file_type not in file_types:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removing: {file_path}, its not in the allowed list: {file_types}")

            # Remove files with "ind" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "ind" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removing ind files (keeping con only): {file_path}")

            # Remove files with "itr_cia_aberta_20" in their names
            for root, _, files in os.walk(zip_subdir_path):
                for file_name in files:
                    if "itr_cia_aberta_20" in file_name:
                        file_path = os.path.join(root, file_name)
                        os.remove(file_path)
                        print(f"Removing itr_cia_aberta_20 file: {file_path}")

            # Convert CSV files from ISO-8859-1 to UTF-8 only if they are not already UTF-8
            for csv_file_name in os.listdir(zip_subdir_path):
                if csv_file_name.endswith(".csv"):
                    csv_file_path = os.path.join(zip_subdir_path, csv_file_name)
                    if not is_utf8(csv_file_path):
                        print(f"Converting encoding: {csv_file_path}")
                        with open(csv_file_path, 'r', encoding='ISO-8859-1') as source_file:
                            content = source_file.read()
                        with open(csv_file_path, 'w', encoding='utf-8') as target_file:
                            target_file.write(content)
                    else:
                        print(f"File already in UTF-8: {csv_file_path}")

print("Download, extraction, file removal, and encoding conversion completed.")


Downloading: itr_cia_aberta_2012.zip
Downloading: itr_cia_aberta_2013.zip
Downloading: itr_cia_aberta_2014.zip
Downloading: itr_cia_aberta_2015.zip
Downloading: itr_cia_aberta_2016.zip
Downloading: itr_cia_aberta_2017.zip
Downloading: itr_cia_aberta_2018.zip
Downloading: itr_cia_aberta_2019.zip
Downloading: itr_cia_aberta_2020.zip
Downloading: itr_cia_aberta_2021.zip
Downloading: itr_cia_aberta_2022.zip
Unzipping: itr_cia_aberta_2012.zip -> downloaded_files/itr_cia_aberta_2012
CSV type: 2012
Removing: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_2012.csv, its not in the allowed list: ['BPA', 'BPP', 'DRE', 'DVA']
CSV type: BPA
CSV type: BPA
CSV type: BPP
CSV type: BPP
CSV type: DFC
Removing: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_DFC_MD_con_2012.csv, its not in the allowed list: ['BPA', 'BPP', 'DRE', 'DVA']
CSV type: DFC
Removing: downloaded_files/itr_cia_aberta_2012/itr_cia_aberta_DFC_MD_ind_2012.csv, its not in the allowed list: ['BPA', 'BPP', 'DRE', 'DVA']
CSV typ

### Creating the datasets

In [ ]:
bpa_df = None
bpp_df = None
dre_df = None
dva_df = None

d_parser = lambda x: pd.datetime.strptime(format='%Y-%m-%d')

# Load files into Data Frames
for dir_name in os.listdir(download_dir):
    if not dir_name.endswith(".zip"):
        dir_path = os.path.join(download_dir, dir_name)

        for csv_file_name in os.listdir(dir_path):
            csv_file_path = os.path.join(dir_path, csv_file_name)
            print("\n-------------------------------------------------------")
            print(f"Reading csv file: {csv_file_path}")

            csv_file_type = os.path.splitext(csv_file_name)[0][15:19].replace("_", "")
            print(f"CSV type: {csv_file_type}")

            local_df = pd.read_csv(csv_file_path, sep=';')

            local_df['DT_REFER'] = pd.to_datetime(local_df['DT_REFER'], format='%Y-%m-%d')
            local_df['DT_FIM_EXERC'] = pd.to_datetime(local_df['DT_FIM_EXERC'], format='%Y-%m-%d')

            print(f"Dataframe size: {len(local_df)}")

            match csv_file_type:
                case 'BPA':
                    if bpa_df is None:
                        bpa_df = local_df
                    else:
                        bpa_df = pd.concat([bpa_df, local_df])
                    print(f"Dataframe size - after concat: {len(bpa_df)}")

                case 'BPP':
                    if bpp_df is None:
                        bpp_df = local_df
                    else:
                        bpp_df = pd.concat([bpp_df, local_df])
                    print(f"Dataframe size - after concat: {len(bpp_df)}")

                case 'DRE':
                    if dre_df is None:
                        dre_df = local_df
                    else:
                        dre_df = pd.concat([dre_df, local_df])
                    print(f"Dataframe size - after concat: {len(dre_df)}")

                case 'DVA':
                    if dva_df is None:
                        dva_df = local_df
                    else:
                        dva_df = pd.concat([dva_df, local_df])
                    print(f"Dataframe size - after concat: {len(dva_df)}")

print("\nbpa_df:")
print(bpa_df.head(2))

print("\nbpp_df:")
print(bpp_df.head(2))

print("\ndre_df:")
print(dre_df.head(2))

print("\ndva_df:")
print(dva_df.head(2))


### Saving Dataframes

In [ ]:
print("\n")
def saveToFile(dfName, df):
    fileName = dfName + '.csv'
    df = df.set_index('DT_REFER')
    print(f'\nsaving file: {fileName}')
    print(df.head(2))
    df.to_csv('processed_files/' + fileName, sep=';')

saveToFile('bpa_df', bpa_df)
saveToFile('bpp_df', bpp_df)
saveToFile('dre_df', dre_df)
saveToFile('dva_df', dva_df)

## 2. Data analising

### Loading Dataframes

In [ ]:
print("\n")
def readCsv(dfName):
    fileName = dfName + '.csv'
    print(f'\nreading file: {fileName}')
    df_read = pd.read_csv('processed_files/' + fileName, sep=';')
    print(df_read.head(2))
    return df_read

bpa_df = readCsv('bpa_df')
bpp_df = readCsv('bpp_df')
dre_df = readCsv('dre_df')
dva_df = readCsv('dva_df')

### Describing data structures

In [ ]:
def print_df(df_name, df):
    print(f'\n\n{df_name}: {len(df):,} records')
    print('-------------------------------------------------------------------')
    print('\t\t\t TYPES')
    print(df.dtypes)

    print('\t\t\t NULL VALUES')
    print('CNPJ_CIA: ' + str(df['CNPJ_CIA'].isnull().sum().sum()))
    print('DT_REFER: ' + str(df['DT_REFER'].isnull().sum().sum()))
    print('VERSAO: ' + str(df['VERSAO'].isnull().sum().sum()))
    print('DENOM_CIA: ' + str(df['DENOM_CIA'].isnull().sum().sum()))
    print('CD_CVM: ' + str(df['CD_CVM'].isnull().sum().sum()))
    print('GRUPO_DFP: ' + str(df['GRUPO_DFP'].isnull().sum().sum()))
    print('MOEDA: ' + str(df['MOEDA'].isnull().sum().sum()))
    print('ESCALA_MOEDA: ' + str(df['ESCALA_MOEDA'].isnull().sum().sum()))
    print('ORDEM_EXERC: ' + str(df['ORDEM_EXERC'].isnull().sum().sum()))
    print('DT_FIM_EXERC: ' + str(df['DT_FIM_EXERC'].isnull().sum().sum()))
    print('CD_CONTA: ' + str(df['CD_CONTA'].isnull().sum().sum()))
    print('DS_CONTA: ' + str(df['DS_CONTA'].isnull().sum().sum()))
    print('VL_CONTA: ' + str(df['VL_CONTA'].isnull().sum().sum()))
    print('ST_CONTA_FIXA: ' + str(df['ST_CONTA_FIXA'].isnull().sum().sum()))

    print('\t\t\t COUNT UNIQUE VALUES')
    print('CNPJ_CIA: ' + str(len(df['CNPJ_CIA'].unique())))
    print('DT_REFER: ' + str(len(df['DT_REFER'].unique())))
    print('VERSAO: ' + str(len(df['VERSAO'].unique())))
    print('DENOM_CIA: ' + str(len(df['DENOM_CIA'].unique())))
    print('CD_CVM: ' + str(len(df['CD_CVM'].unique())))
    print('GRUPO_DFP: ' + str(len(df['GRUPO_DFP'].unique())))
    print('MOEDA: ' + str(len(df['MOEDA'].unique())))
    print('ESCALA_MOEDA: ' + str(len(df['ESCALA_MOEDA'].unique())))
    print('ORDEM_EXERC: ' + str(len(df['ORDEM_EXERC'].unique())))
    print('DT_FIM_EXERC: ' + str(len(df['DT_FIM_EXERC'].unique())))
    print('CD_CONTA: ' + str(len(df['CD_CONTA'].unique())))
    print('DS_CONTA: ' + str(len(df['DS_CONTA'].unique())))
    print('VL_CONTA: ' + str(len(df['VL_CONTA'].unique())))
    print('ST_CONTA_FIXA: ' + str(len(df['ST_CONTA_FIXA'].unique())))

    print('\t\t\t UNIQUE VALUES')
    print('CNPJ_CIA: ' + str(len(df['CNPJ_CIA'].unique())))
    print('DT_REFER: ' + str(len(df['DT_REFER'].unique())))
    print('VERSAO: ' + str(df['VERSAO'].unique()))
    print('DENOM_CIA: ' + str(len(df['DENOM_CIA'].unique())))
    print('CD_CVM: ' + str(len(df['CD_CVM'].unique())))
    print('GRUPO_DFP: ' + str(df['GRUPO_DFP'].unique()))
    print('MOEDA: ' + str(df['MOEDA'].unique()))
    print('ESCALA_MOEDA: ' + str(df['ESCALA_MOEDA'].unique()))
    print('ORDEM_EXERC: ' + str(df['ORDEM_EXERC'].unique()))
    print('DT_FIM_EXERC: ' + str(len(df['DT_FIM_EXERC'].unique())))
    print('CD_CONTA: ' + str(len(df['CD_CONTA'].unique())))
    print('DS_CONTA: ' + str(len(df['DS_CONTA'].unique())))
    print('VL_CONTA: ' + str(len(df['VL_CONTA'].unique())))
    print('ST_CONTA_FIXA: ' + str(df['ST_CONTA_FIXA'].unique()))


print_df('bpa_df', bpa_df)
print_df('bpp_df', bpp_df)
print_df('dre_df', dre_df)
print_df('dva_df', dva_df)


## 3. Data cleansing

### Cleaning up

In [ ]:
# calculate value according to field ESCALA_MOELA
def set_vl_conta(df_name, df):
    print(f'setting vl_conta based on escala_moeda: {df_name}')
    df['VL_CONTA'] = np.where(df['ESCALA_MOEDA'] == 'MIL', df['VL_CONTA'] * 1000, df['VL_CONTA'])
    return df

def remove_non_mandatory_columns(df_name, df):
    print(f'removing non mandatory columns from: {df_name}')
    existing_cols = df.columns.intersection(['CNPJ_CIA', 'VERSAO', 'DENOM_CIA', 'MOEDA', 'ESCALA_MOEDA', 'DT_INI_EXERC', 'COLUNA_DF', 'ST_CONTA_FIXA'])
    return df.drop(columns=existing_cols, axis=1)

def remove_non_used_accounts(df_name, df):
    print(f'removing non used accounts for: {df_name}')
    print(f'accounts selected: {account_to_be_kept}')
    return df[df['CD_CONTA'].isin(account_to_be_kept)]


print("\n")
bpa_df = set_vl_conta('bpa_df', bpa_df)
bpp_df = set_vl_conta('bpp_df', bpp_df)
dre_df = set_vl_conta('dre_df', dre_df)
dva_df = set_vl_conta('dva_df', dva_df)

print("\n")
bpa_df = remove_non_mandatory_columns('bpa_df', bpa_df)
bpp_df = remove_non_mandatory_columns('bpp_df', bpp_df)
dre_df = remove_non_mandatory_columns('dre_df', dre_df)
dva_df = remove_non_mandatory_columns('dva_df', dva_df)

print("\n")
bpa_df = remove_non_used_accounts('bpa_df', bpa_df)
bpp_df = remove_non_used_accounts('bpp_df', bpp_df)
dre_df = remove_non_used_accounts('dre_df', dre_df)
dva_df = remove_non_used_accounts('dva_df', dva_df)


## 4. Data transformation

### Merge dataframes into one

In [ ]:
df = pd.concat([bpa_df, bpp_df, dre_df, dva_df])

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()

### Saving merged Dataframes

In [ ]:
saveToFile('df', df)


### Loading merged Dataframes

In [ ]:
df = readCsv('df')

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()

### Selecting companies

In [ ]:
print(f'selecting companies : {companies}')
companies_df = df[df['CD_CVM'].isin(companies)]

print(companies_df.dtypes)
print("\n")
print(companies_df)
print("\n")
companies_df.head(2)

### Saving companies Dataframe

In [ ]:
saveToFile('../companies_df', companies_df)


### Loading companies DataFrame

In [ ]:
df = readCsv('../companies_df')

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()

### Basic functions for cleaning up

In [ ]:
# Reorder the columns to move 'ORDEM_EXERC' to the last position
cols = list(df.columns)
cols.remove('ORDEM_EXERC')
cols.insert(-1, 'ORDEM_EXERC')
df = df[cols]

# Reorder the columns to move 'CD_CONTA' to the 2nd position
cols = list(df.columns)
cols.remove('CD_CONTA')
cols.insert(2, 'CD_CONTA')
df = df[cols]

# DT_REFER asdatetime
df['DT_REFER'] = pd.to_datetime(df['DT_REFER'], format='%Y-%m-%d')


def createPenultimo_DT_FIM_EXERC(df, position):

    columnName = 'DT_FIM_EXERC'

    # The field DT_FIM_EXERC for BPA and BPP is 3 months in the past, for the others (DRE, DVA) this value is past 12 months
    newColumnNameBP = columnName + '_PENULTIMO' + '_BP'
    newColumnName = columnName + '_PENULTIMO'
    bpList = ['DF Consolidado - Balanço Patrimonial Ativo', 'DF Consolidado - Balanço Patrimonial Passivo']

    df[newColumnNameBP] = np.where(
        (df['ORDEM_EXERC'] == 'PENÚLTIMO') & df['GRUPO_DFP'].isin(bpList),
        df[columnName],
        None)
    df[newColumnName] = np.where(
        (df['ORDEM_EXERC'] == 'PENÚLTIMO') & ~(df['GRUPO_DFP'].isin(bpList)),
        df[columnName],
        None)

    # remove this same values from column
    df[columnName] = np.where(
        df['ORDEM_EXERC'] != 'PENÚLTIMO',
        df[columnName],
        None)

    df[newColumnNameBP] = pd.to_datetime(df[newColumnNameBP], format='%Y-%m-%d')
    df[newColumnName] = pd.to_datetime(df[newColumnName], format='%Y-%m-%d')
    df[columnName] = pd.to_datetime(df[columnName], format='%Y-%m-%d')

    # Reorder the columns to move to correct position
    cols = list(df.columns)

        # BP
    cols.remove(newColumnNameBP)
    cols.insert(position, newColumnNameBP)
    df = df[cols]

        # not BP
    cols.remove(newColumnName)
    cols.insert(position+1, newColumnName)
    df = df[cols]

    return df

def createPenultimo_VL_CONTA(df, position):

    columnName = 'VL_CONTA'

    # The field DT_FIM_EXERC for BPA and BPP is 3 months in the past, for the others (DRE, DVA) this value is past 12 months
    newColumnName = columnName + '_PENULTIMO'

    df[newColumnName] = np.where(
        (df['ORDEM_EXERC'] == 'PENÚLTIMO'),
        df[columnName],
        None)

    # remove this same values from column
    df[columnName] = np.where(
        df['ORDEM_EXERC'] != 'PENÚLTIMO',
        df[columnName],
        None)

    df[newColumnName] = df[newColumnName].astype(float)
    df[columnName] = df[columnName].astype(float)

    # Reorder the columns to move to correct position
    cols = list(df.columns)

    cols.remove(newColumnName)
    cols.insert(position, newColumnName)
    df = df[cols]

    return df


### Create column 'DT_FIM_EXERC_PENULTIMO'

In [ ]:
df = createPenultimo_DT_FIM_EXERC(df, 4)

print(df.dtypes)
print("\n")
df.head()


#### Create column 'VL_CONTA_PENULTIMO'

In [ ]:
df = createPenultimo_VL_CONTA(df, 8)

print(df.dtypes)
print("\n")
df.head()

#### Remove disposable columns

In [ ]:
cols = df.columns.intersection(['GRUPO_DFP', 'DS_CONTA', 'ORDEM_EXERC'])
df = df.drop(columns=cols, axis=1)

print(df.dtypes)
print("\n")
df.head()

#### Group by 'DT_REFER', 'CD_CVM', and 'CD_CONTA' and aggregate the values

In [ ]:
agg_funcs = {
    'DT_FIM_EXERC_PENULTIMO_BP': 'first',
    'DT_FIM_EXERC_PENULTIMO': 'first',
    'DT_FIM_EXERC': 'first',
    'VL_CONTA_PENULTIMO': 'last',
    'VL_CONTA': 'last'
}

df = df.groupby(['DT_REFER', 'CD_CVM', 'CD_CONTA']).agg(agg_funcs).reset_index()

# Reorder the columns as needed
df = df[['DT_REFER', 'CD_CVM', 'CD_CONTA', 'DT_FIM_EXERC_PENULTIMO_BP', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC', 'VL_CONTA_PENULTIMO', 'VL_CONTA']]

# View the final DataFrame
print(df.dtypes)
print("\n")
df.head()

#### Saving df before transposing

In [ ]:
saveToFile('../companies_before_transposing_df', df)


#### Loading df before transposing

In [ ]:
df = readCsv('../companies_before_transposing_df')

print(df.dtypes)
print("\n")
print(df)
print("\n")
df.head()


#### Transforming rows into column to have one register by date, company and account type

In [ ]:
# Filling NaN values with empty strings
df.fillna('', inplace=True)

# Creating the pivot table
df_pivot = df.pivot_table(
    index=['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC_PENULTIMO_BP', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'],
    columns='CD_CONTA',
    values=['VL_CONTA_PENULTIMO', 'VL_CONTA'],
    aggfunc='first'
).reset_index()

# Flattening the multi-level columns
df_pivot.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_pivot.columns.values]

# Reordering the columns
ordered_columns = (
    ['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC_PENULTIMO_BP', 'DT_FIM_EXERC_PENULTIMO', 'DT_FIM_EXERC'] +
    [col for col in df_pivot.columns if col.startswith('VL_CONTA_PENULTIMO_')] +
    [col for col in df_pivot.columns if col.startswith('VL_CONTA_') and not col.startswith('VL_CONTA_PENULTIMO_')]
)

df_pivot = df_pivot[ordered_columns]

print("\n-------------------------------------")
print(df_pivot)
print("-------------------------------------\n")

def custom_agg(series):
    # If the dtype is datetime, check for non-NaT values
    if series.dtype == 'datetime64[ns]':
        non_nat_values = [val for val in series if not pd.isna(val)]
        return non_nat_values[0] if non_nat_values else pd.NaT
    else:
        # If it's numeric, check for non-NaN values
        if pd.api.types.is_numeric_dtype(series):
            non_nan_values = [val for val in series if not pd.isna(val)]
            return non_nan_values[0] if non_nan_values else np.nan
        # Else check for non-empty strings
        else:
            non_empty_values = [val for val in series if val != '']
            return non_empty_values[0] if non_empty_values else ''

# Create an aggregation dictionary excluding columns you're grouping by
grouping_columns = ['DT_REFER', 'CD_CVM', 'DT_FIM_EXERC']
agg_dict = {col: custom_agg for col in ordered_columns if col not in grouping_columns}

# Group by 'DT_REFER', 'CD_CVM', and 'DT_FIM_EXERC', then aggregate
grouped_df = df_pivot.groupby(grouping_columns).agg(agg_dict).reset_index()

cols = list(grouped_df.columns)
cols.remove('DT_FIM_EXERC')
cols.insert(4, 'DT_FIM_EXERC')
grouped_df = grouped_df[cols]

print("\n-------------------------------------")
print(grouped_df)
print("-------------------------------------\n")

saveToFile('../df_pivot', grouped_df)


#### Reading the cleaned up and grouped dataframe

In [ ]:
final_df = readCsv('../df_pivot')

print(final_df.dtypes)
print("\n")
print(final_df)
print("\n")
final_df.head()

#### Free Cash Flow formula

*FCFE = <br>
  3.11 (Lucro/Prejuizo Consolidado do Periodo do DRE)<br>
  \- (Δ 1.02.03 (Imobilizado) - 7.04.01 (Depreciacao, Amortizacao e Exaustao da DVA))<br>
  \- (Δ 1.01 (Ativo Circulante) - Δ 1.01.01 (Caixa e Equivalentes de Caixa) -  Δ 2.01 (Passivo Circulante))<br>
  \+ (Δ 2.01.04(Emprestimos e Financiamentos Circulantes) + Δ 2.02.01(Emprestimos e Financiamentos Nao Circulantes))*<br><br>


*3.11 (Lucro/Prejuizo Consolidado do Periodo do DRE) = <br>
  LUCRO_LIQUIDO_PERIODO*<br>


*Δ 1.02.03 (Imobilizado) = <br>
  IMOBILIZADO_NO_PERIODO_ANTERIOR - IMOBILIZADO_NO_PERIODO*<br>


*7.04.01 (Depreciacao, Amortizacao e Exaustao da DVA) = <br>
  DEPRECIACAO_AMORTIZACAO_E_EXAUSTAO_DA_DVA*<br>


*Δ 1.01 (Ativo Circulante) = <br>
  ATIVO_CIRCULANTE_NO_PERIODO_ANTERIOR - ATIVO_CIRCULANTE_NO_PERIODO*<br>


*Δ 1.01.01 (Caixa e Equivalentes de Caixa) = <br>
  CAIXA_E_EQUIVALENTES_DE_CAIXA_NO_PERIODO_ANTERIOR - CAIXA_E_EQUIVALENTES_DE_CAIXA_NO_PERIODO*<br>


*Δ 2.01 (Passivo Circulante) = <br>
  PASSIVO_CIRCULANTE_NO_PERIODO_ANTERIOR - PASSIVO_CIRCULANTE_NO_PERIODO*<br>


*Δ 2.01.04(Emprestimos e Financiamentos Circulantes) = <br>
  EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES_NO_PERIODO_ANTERIOR - EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES_NO_PERIODO*<br>


*Δ 2.02.01(Emprestimos e Financiamentos Nao Circulantes) = <br>
  EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES_NO_PERIODO_ANTERIOR - EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES_NO_PERIODO*<br>

In [ ]:
####################################################################
# Lucro/Prejuizo Consolidado do Periodo do DRE (3.11)
####################################################################
final_df['LUCRO_LIQUIDO_PERIODO'] = final_df['VL_CONTA_3.11']


####################################################################
# Δ Imobilizado (1.02.03)
####################################################################
final_df['IMOBILIZADO_NO_PERIODO_ANTERIOR'] = final_df['VL_CONTA_PENULTIMO_1.02.03']
final_df['IMOBILIZADO_NO_PERIODO'] = final_df['VL_CONTA_1.02.03']
final_df['IMOBILIZADO'] = final_df['IMOBILIZADO_NO_PERIODO_ANTERIOR'] - final_df['IMOBILIZADO_NO_PERIODO']


#################################################################### 
# Depreciacao, Amortizacao e Exaustao da DVA (7.04.01)
#################################################################### 
final_df['DEPRECIACAO_AMORTIZACAO_E_EXAUSTAO_DA_DVA'] = final_df['VL_CONTA_7.04.01']


#################################################################### 
# Δ Ativo Circulante (1.01)
#################################################################### 
final_df['ATIVO_CIRCULANTE_NO_PERIODO_ANTERIOR'] = final_df['VL_CONTA_PENULTIMO_1.01']
final_df['ATIVO_CIRCULANTE_NO_PERIODO'] = final_df['VL_CONTA_1.01']
final_df['ATIVO_CIRCULANTE'] = final_df['ATIVO_CIRCULANTE_NO_PERIODO_ANTERIOR'] - final_df['ATIVO_CIRCULANTE_NO_PERIODO']


#################################################################### 
# Δ Caixa e Equivalentes de Caixa (1.01.01)
#################################################################### 
final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA_NO_PERIODO_ANTERIOR'] = final_df['VL_CONTA_PENULTIMO_1.01.01']
final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA_NO_PERIODO'] = final_df['VL_CONTA_1.01.01']
final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA'] = final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA_NO_PERIODO_ANTERIOR'] - final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA_NO_PERIODO']


#################################################################### 
# Δ Passivo Circulante (2.01)
#################################################################### 
final_df['EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES_NO_PERIODO_ANTERIOR'] = final_df['VL_CONTA_PENULTIMO_2.01']
final_df['EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES_NO_PERIODO'] = final_df['VL_CONTA_2.01']
final_df['EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES'] = final_df['EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES_NO_PERIODO_ANTERIOR'] - final_df['EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES_NO_PERIODO']


#################################################################### 
# Δ Emprestimos e Financiamentos Circulantes (2.01.04)
#################################################################### 
final_df['PASSIVO_CIRCULANTE_NO_PERIODO_ANTERIOR'] = final_df['VL_CONTA_PENULTIMO_2.01.04']
final_df['PASSIVO_CIRCULANTE_NO_PERIODO'] = final_df['VL_CONTA_2.01.04']
final_df['PASSIVO_CIRCULANTE'] = final_df['PASSIVO_CIRCULANTE_NO_PERIODO_ANTERIOR'] - final_df['PASSIVO_CIRCULANTE_NO_PERIODO']


#################################################################### 
# Δ Emprestimos e Financiamentos Nao Circulantes (2.02.01)
#################################################################### 
final_df['EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES_NO_PERIODO_ANTERIOR'] = final_df['VL_CONTA_PENULTIMO_2.02.01']
final_df['EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES_NO_PERIODO'] = final_df['VL_CONTA_2.02.01']
final_df['EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES'] = final_df['EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES_NO_PERIODO_ANTERIOR'] - final_df['EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES_NO_PERIODO']


# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#
# Fluxo de Caixa Livre para os Acionistas (Free Cash Flows to Equity) - FCFE
#
# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
final_df['FCFE'] = \
    final_df['LUCRO_LIQUIDO_PERIODO'] \
    - ( final_df['IMOBILIZADO'] - final_df['DEPRECIACAO_AMORTIZACAO_E_EXAUSTAO_DA_DVA'] ) \
    - ( final_df['ATIVO_CIRCULANTE'] - final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA'] - final_df['EMPRESTIMOS_E_FINANCIAMENTOS_CIRCULANTES'] ) \
    + ( final_df['PASSIVO_CIRCULANTE'] + final_df['CAIXA_E_EQUIVALENTES_DE_CAIXA'] - final_df['EMPRESTIMOS_E_FINANCIAMENTOS_NAO_CIRCULANTES'] )

saveToFile('../final_df', final_df)

print(final_df)

#### Loading the data already strutured and cleaned

In [ ]:
# ARIMA
%pip install statsmodels
# matplotlib (para visualizações)
%pip install matplotlib
# pmdarima (para a seleção automática do modelo ARIMA)
%pip install pmdarima
# statsmodels
%pip install statsmodels

In [ ]:
import pandas as pd

data = pd.read_csv('final_df.csv', sep=';')

data['DT_REFER'] = pd.to_datetime(data['DT_REFER'], format='%Y-%m-%d')
data['DT_FIM_EXERC_PENULTIMO_BP'] = pd.to_datetime(data['DT_FIM_EXERC_PENULTIMO_BP'], format='%Y-%m-%d')
data['DT_FIM_EXERC_PENULTIMO'] = pd.to_datetime(data['DT_FIM_EXERC_PENULTIMO'], format='%Y-%m-%d')
data['DT_FIM_EXERC'] = pd.to_datetime(data['DT_FIM_EXERC'], format='%Y-%m-%d')
data = data.set_index('DT_REFER')


#### Exploratory Data Analysis (EDA)

##### Time Series Visualization

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

def thousands_formatter(x, pos):
    return f'{int(x):,}'

formatter = FuncFormatter(thousands_formatter)

def print_serie(df, company_code):
    company = companies[company_code]
    plt.figure(figsize=(14,7))
    plt.plot(df.index, df['FCFE'])
    plt.title(f"Fluxo de Caixa Livre ao Longo do Tempo para a empresa: {company['name']} ({company['ticker']})")
    plt.xlabel('Data')
    plt.ylabel('Fluxo de Caixa Livre')
    plt.grid(True)
    plt.ticklabel_format(style='plain', axis='y')
    ax = plt.gca()  # get current axis
    ax.yaxis.set_major_formatter(formatter)  # set the formatter for y axis
    plt.show()

for company_code in companies:
    company_data = data[data['CD_CVM'] == company_code]
    print_serie(company_data, company_code)


def plot_combined_series(data, companies):
    plt.figure(figsize=(14,7))
    
    for company_code in companies:
        company_data = data[data['CD_CVM'] == company_code]
        company = companies[company_code]
        plt.plot(company_data.index, company_data['FCFE'], label=f"{company['name']} ({company['ticker']})")
    
    plt.title('Fluxo de Caixa Livre ao Longo do Tempo')
    plt.xlabel('Data')
    plt.ylabel('Fluxo de Caixa Livre')
    plt.legend(loc='best')
    plt.ticklabel_format(style='plain', axis='y')
    ax = plt.gca()  # get current axis
    ax.yaxis.set_major_formatter(formatter)  # set the formatter for y axis
    plt.grid(True)
    plt.show()

plot_combined_series(data, companies)

##### Time Series Decomposition

In [ ]:
import statsmodels.api as sm
import matplotlib.ticker as ticker

def decompose_series(data, period=12):
    decomposition = sm.tsa.seasonal_decompose(data, period=period)
    return decomposition

def plot_individual_decomposition(data, company_code, period=12):
    company = companies[company_code]
    company_data = data[data['CD_CVM'] == company_code]

    decomposition = decompose_series(company_data['FCFE'], period)

    # Thousand separator
    formatter = ticker.FuncFormatter(lambda x, p: format(int(x), ','))

    plt.figure(figsize=(14, 7))
    plt.subplot(3, 1, 1)
    plt.plot(decomposition.trend)
    plt.title(f"Tendência - {company['name']} ({company['ticker']})")
    plt.gca().yaxis.set_major_formatter(formatter)

    plt.subplot(3, 1, 2)
    plt.plot(decomposition.seasonal)
    plt.title(f"Sazonalidade - {company['name']} ({company['ticker']})")
    plt.gca().yaxis.set_major_formatter(formatter)

    plt.subplot(3, 1, 3)
    plt.plot(decomposition.resid)
    plt.title(f"Resíduo - {company['name']} ({company['ticker']})")
    plt.gca().yaxis.set_major_formatter(formatter)

    plt.tight_layout()
    plt.show()

for company_code in companies:
    plot_individual_decomposition(data, company_code)



def plot_grouped_decomposition(data, companies, period=12):
    plt.figure(figsize=(14, 21))

    # Thousand separator
    formatter = ticker.FuncFormatter(lambda x, p: format(int(x), ','))

    for company_code in companies:
        company_data = data[data['CD_CVM'] == company_code]
        company = companies[company_code]

        # Serie decomposition
        decomposition = decompose_series(company_data['FCFE'], period)

        # Plotting Tendency
        plt.subplot(3, 1, 1)
        plt.plot(decomposition.trend, label=f"{company['name']} ({company['ticker']})")
        plt.title('Tendência Agrupada')
        plt.legend(loc='best')
        plt.gca().yaxis.set_major_formatter(formatter)

        # Plotting Tendency Seasonality
        plt.subplot(3, 1, 2)
        plt.plot(decomposition.seasonal, label=f"{company['name']} ({company['ticker']})")
        plt.title('Sazonalidade Agrupada')
        plt.legend(loc='best')
        plt.gca().yaxis.set_major_formatter(formatter)

        # Plotting Resid
        plt.subplot(3, 1, 3)
        plt.plot(decomposition.resid, label=f"{company['name']} ({company['ticker']})")
        plt.title('Resíduo Agrupado')
        plt.legend(loc='best')
        plt.gca().yaxis.set_major_formatter(formatter)

    plt.tight_layout()
    plt.show()

plot_grouped_decomposition(data, companies)

##### Stationarity validation

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test_and_check(data):
    # Performing the Augmented Dickey-Fuller test:
    result = adfuller(data)
    
    # Printing results:
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])
    
    # Checking the p-value to decide on stationarity:
    if result[1] <= 0.05:
        print("Com 95% de confiança, podemos dizer que a série é estacionária.")
        return True
    else:
        print("Com 95% de confiança, podemos dizer que a série NÃO é estacionária.")
        return False


stationary_series_dict = {}
non_stationary_companies = {}

for company_code in companies:
    company_info = companies[company_code]
    company_data = data[data['CD_CVM'] == company_code]
    print("\n\n------------------------------------------------------")
    print(f"Resultado para a empresa {company_info['name']} ({company_info['ticker']})")
    print("------------------------------------------------------")
    is_stationary = adfuller_test_and_check(company_data['FCFE'])
    if is_stationary:
        stationary_series_dict[company_code] = company_data['FCFE']
    else:
        non_stationary_companies[company_code] = company_data['FCFE']


##### Non-stationarity transformations

In [ ]:
def make_series_stationary(series, max_diff=2):
    diff_count = 0
    stationary_series = series.copy()
    
    while diff_count < max_diff:
        diff_count += 1
        stationary_series = stationary_series.diff().dropna()  # Differencing the series
        
        print("\n...............................................")
        print("Checking stationarity after differentiation")
        print("...............................................")
        is_stationary = adfuller_test_and_check(stationary_series)

        if is_stationary:
            print(f"\n=> Série tornou-se estacionária após {diff_count} diferenciação(ões).")
            return stationary_series
        
    print(f"\n=> A série não se tornou estacionária após {max_diff} diferenciações.")
    return None  # Returns None if unable to make the series stationary after max_diff attempts

# Using the non_stationary_companies dictionary to transform the non-stationary series
for company_code in non_stationary_companies:
    company_info = companies[company_code]
    company_data = data[data['CD_CVM'] == company_code]
    print("\n\n------------------------------------------------------")
    print(f"Transformando série para a empresa {company_info['name']} ({company_info['ticker']})")
    print("------------------------------------------------------")
    
    # Making the series stationary
    stationary_series = make_series_stationary(company_data['FCFE'])
    if stationary_series is not None:
        stationary_series_dict[company_code] = stationary_series


#### Prediction using ARIMA